**Process the audio file 0.1 sec by 0.1 sec. In each iteration, create a spectrogram and predict whether it is an "ah" or not. Save all the predictions in a list. Find out the time locations of all the "ah-s" and cut the initial audio there.**

In [1]:
from fastai.vision.all import *
from fastbook import *

C:\Users\Dicey\anaconda3\envs\audio\lib\site-packages\fastbook\__init__.py:10: UserWarning: Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`
  except ModuleNotFoundError: warn("Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`")
C:\Users\Dicey\anaconda3\envs\audio\lib\site-packages\fastbook\__init__.py:15: UserWarning: Missing Azure SDK - please run `pip install azure-cognitiveservices-search-imagesearch`
  warn("Missing Azure SDK - please run `pip install azure-cognitiveservices-search-imagesearch`")


In [2]:
load_path = os.path.join('./images/models/model1.pth')

def is_ah(x) : return x.parent.name
path = os.path.join('./images/')
dls = ImageDataLoaders.from_path_func(
    path, get_image_files(path), num_workers=0 , valid_pct=0.2, seed=42,
    label_func=is_ah) # added num_workers = 0

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.load('model1', device='cuda')

RuntimeError: Error(s) in loading state_dict for Sequential:
	size mismatch for 1.8.weight: copying a param with shape torch.Size([2, 512]) from checkpoint, the shape in current model is torch.Size([3, 512]).

Check the predictions by checking one single file

In [ ]:
uploader = widgets.FileUpload()
uploader

In [ ]:
img = PILImage.create(uploader.data[0])
print(type(uploader.data[0]))
print(type(img))
is_ah,_,probs = learn.predict(img)
print(f"Is this an AH?: {is_ah}.")
print(f"Probability it's a AH: {probs[1].item():.6f}")

In [ ]:
img_path = os.path.join('./images/1/')


**Now process a whole piece of audio**

In [ ]:
import librosa
import matplotlib.pyplot as plt
import os
import glob
from pydub import AudioSegment
from pydub.utils import make_chunks
import io
from PIL import Image

In [ ]:
audio_path = os.path.join('./audio_full/test1.wav')
audio_full = librosa.load(audio_path, sr=16000) # load and convert to 16 khz

Process the audio file 0.1 sec by 0.1 sec. In each iteration, create a spectrogram and predict whether it is an "ah" or not. Save all the predictions in a list. Find out the time locations of all the "ah-s" and cut the initial audio there.

In [ ]:
len(audio_full[0])/16000/60 # this is the length of the audio in  minutes (16000 samples/amplitude values per second)

For every 1600 samples I need to create a spectrogram and pass it through the prediction algorithm

**Resources** 

[Converting a matplotlib figure into bytes](https://stackoverflow.com/questions/8598673/how-to-save-a-pylab-figure-into-in-memory-file-which-can-be-read-into-pil-image/8598881) and then into PILimage

Process the audio file 0.1 sec by 0.1 sec. In each iteration, create a spectrogram and predict whether it is an "ah" or not. Save all the predictions in a list.

In [ ]:
step = 1600 # 0.1 second steps
count = 0
timeline = []

# change the second number '320000' to adjust how long a piece of processed audio should be
for audio_chunk in audio_full[0][::1600] :
    count = count + 1
    stop = count * step # 1600, 3200, 4800, ...
    start = stop - step # 0, 1600, 3200, ...
    audio = audio_full[0][start:stop]
    
    plt.figure(figsize=(0.97,1), dpi=298)
    plt.specgram(audio, Fs=16000, NFFT=32, noverlap=16) # create a spectrogram
    plt.gca().set_axis_off() # remove the axis text
    # THE PROBLEM: All the predictions are 0.
    # Reason? Not sure if plots that are converted to bytes still have white space around which makes them unrecognizable 
    # for the model.
    
    # convert the figure to bytes
    buf = io.BytesIO()
    plt.savefig(buf, format='jpg', dpi=298, pad_inches=0, bbox_inches='tight')
    buf.seek(0)
    
    # convert the image from bytes to PIL
    img = PILImage.create(buf)
    # show the probability for this 0.1 piece of audio to be an "ah"
    is_ah,_,probs = learn.predict(img)
    timeline.append(is_ah)
    
    plt.close('all')
    buf.close()

In [ ]:
timeline